In [1]:
import pandas as pd
import sqlite3
import pymongo

# 建立類別與方法

In [2]:
class dataToSQL():
    SQL_DB = 'Jobsearch_database'
    NoSQL_DB = 'JobDB'
    columns_num = 23
    columns_name = '更新日期,職缺名稱,公司名稱,工作內容,職務類別,\
                    工作待遇,工作性質,上班地點,管理責任,出差外派,\
                    上班時段,休假制度,可上班日,需求人數,工作經歷,\
                    學歷要求,科系要求,語文條件,擅長工具,工作技能,\
                    其他要求,連結,縣市'
    def __init__(self, tablename = None, collectionname = None):
        if tablename:
            self.tablename = tablename
            self.create_schema()
        if collectionname:
            self.collectionname = collectionname
    def create_schema(self):
        conn = sqlite3.connect("Jobsearch_database.db")
        create_table_query = f'''
        CREATE TABLE IF NOT EXISTS {self.tablename}  (
            '更新日期' Date,
            '職缺名稱' VARCHAR(50),
            '公司名稱' VARCHAR(50),
            '工作內容' TEXT,
            '職務類別' VARCHAR(50),
            '工作待遇' VARCHAR(50),
            '工作性質' VARCHAR(10),
            '上班地點' VARCHAR(50),
            '管理責任' VARCHAR(50),
            '出差外派' VARCHAR(50),
            '上班時段' VARCHAR(50),
            '休假制度' VARCHAR(50),
            '可上班日' VARCHAR(50),
            '需求人數' VARCHAR(50),
            '工作經歷' VARCHAR(50),
            '學歷要求' VARCHAR(50),
            '科系要求' VARCHAR(50),
            '語文條件' VARCHAR(50),
            '擅長工具' VARCHAR(50),
            '工作技能' VARCHAR(50),
            '其他要求' VARCHAR(50),
            '連結' TEXT PRIMARY KEY,
            '縣市' VARCHAR(10)
        )
        '''
        conn.execute(create_table_query)
        conn.commit()
        conn.close()
    def SQL_replace_data(self, *args):
        placeholders = ','.join(['?'] * self.columns_num)
        conn = sqlite3.connect("Jobsearch_database.db")
        for df in args:
            replace_query = f"REPLACE INTO {self.tablename} ({self.columns_name}) VALUES ({placeholders})"
            cursor = conn.cursor()
            cursor.executemany(replace_query, df.values.tolist())
        conn.commit()
        conn.close()
    
    def NoSQL_replace_data(self, *args):
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        db = client[self.NoSQL_DB]
        collection = db[self.collectionname]
        new_count, update_count = 0, 0
        for df in args:
            data = df.to_dict(orient="records")
            for idx, record in enumerate(data):
                try:
                    filter_query = {"連結": record["連結"]}
                    existing_record = collection.find_one(filter_query)
                    # 已存在就更新;不存在就插入
                    if existing_record is None:
                        new_count += 1
                        collection.insert_one(record)
                    else:
                        update_count += 1
                        collection.replace_one(filter_query, record)
                except Exception as e:
                    print(f'{idx},{e}')
                    continue
        print(f'更新{update_count}筆, 新增{new_count}筆')

# To DataBase

In [7]:
# Load csv (通常一次一個, 每次search完匯入SQL)
file_name = 'JBLIST_2023-08-08.csv'
df = pd.read_csv(f'../output/{file_name}')

In [8]:
test = dataToSQL('DE_jb','jobdata')
test.SQL_replace_data(df)
test.NoSQL_replace_data(df)

更新65筆, 新增313筆
